In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
tf.keras.utils.set_random_seed(42)
import altair as alt

2023-10-22 04:04:35.508924: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-22 04:04:35.548765: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-22 04:04:35.549350: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 04:04:36.340918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# OECD_model = keras.models.load_model('/oecd/NN_OECD_Model.h5')
WB_model = keras.models.load_model('world_bank/NN_WorldBank_Model.h5')

2023-10-22 04:04:37.990009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-22 04:04:37.990497: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
X_train = pd.read_csv('world_bank/X_train.csv')
y_train= pd.read_csv('world_bank/y_train.csv')
X_test= pd.read_csv('world_bank/X_test.csv')
y_test= pd.read_csv('world_bank/y_test.csv')

In [4]:
# y_pred = best_model.predict(X_test)
# This takes the neural network model and passes in X_test and returns predicted values based on the model
y_pred = WB_model.predict(X_test)

8/8 [==============================] - 0s 1ms/step


In [5]:
df_x_test_with_y_actual_y_pred = X_test.copy()
df_x_test_with_y_actual_y_pred['y_test'] = y_test
df_x_test_with_y_actual_y_pred['y_pred'] = y_pred

In [6]:
y_residuals = df_x_test_with_y_actual_y_pred['y_test'] - df_x_test_with_y_actual_y_pred['y_pred']
df_x_test_with_y_actual_y_pred['y_residuals'] = y_residuals
df_x_test_with_y_actual_y_pred['y_residuals_abs_val'] = y_residuals.abs()

# sort the residuals based on absolute values
df_x_test_with_y_actual_y_pred_sorted_by_abs_residuals = df_x_test_with_y_actual_y_pred.sort_values(
    'y_residuals_abs_val', ascending = False)

In [7]:
top_10_outliers_df = df_x_test_with_y_actual_y_pred_sorted_by_abs_residuals.iloc[0:10]

In [8]:
# Make it clear which rows are outliers
df_x_test_with_y_actual_y_pred['is_outlier'] = df_x_test_with_y_actual_y_pred.index.isin(top_10_outliers_df.index)

In [9]:
outlier_df = df_x_test_with_y_actual_y_pred[['y_test', 'y_pred', 'is_outlier']]

In [10]:
# outlier_df

In [11]:
# Create a new DataFrame with y_test and y_pred

# Scatter plot
scatter_actual_vs_predicted = alt.Chart(outlier_df).mark_circle(opacity=0.5).encode(
    x = alt.X('y_test', title= "Actual Maternal Mortality Values (y_test)"),
    y = alt.Y('y_pred', title = "Predicted Maternal Mortality Values (y_pred)"),
    color = alt.Color('is_outlier'),
    tooltip = ['y_test', 'y_pred']
)

# scatter_actual_vs_predicted

# y = x line
y_test_min = min(outlier_df['y_test'].values)
y_test_max = max(outlier_df['y_test'].values)

y_pred_min = min(outlier_df['y_pred'].values)
y_pred_max = max(outlier_df['y_pred'].values)

max_y = max(y_test_max,y_pred_max)

line_dict = {
    'y_test': [0, max_y],
    'y_pred': [0, max_y],
}

line_df = pd.DataFrame(data=line_dict)

y_eq_x_line = alt.Chart(line_df).mark_line(color = 'red',
                                           # strokeDash=[5, 5] makes dashed line
                                           strokeDash=[5, 5]).encode(
    x = alt.X('y_test'),
    y = alt.Y('y_pred')
)

final_chart = scatter_actual_vs_predicted + y_eq_x_line
final_chart_interactive = final_chart.properties(
    width = 800,
    height = 550,
    title = {
        "text": "Actual Maternal Mortality Values (y_test) Vs Predicted Maternal Mortality Values (y_pred)",
        "subtitle": "With top 10 outliers"
    }
)

In [12]:
final_chart_interactive

alt.LayerChart(...)